In [53]:
import numpy as np
import pandas as pd

In [54]:
data = pd.read_csv('weekdays/monday.csv', delimiter = ';', parse_dates=True)

In [55]:
data.head()

,timestamp,customer_no,location
0,2019-09-02 07:03:00,1,dairy
1,2019-09-02 07:03:00,2,dairy
2,2019-09-02 07:04:00,3,dairy
3,2019-09-02 07:04:00,4,dairy
4,2019-09-02 07:04:00,5,spices


In [56]:
data.dtypes

timestamp      object
customer_no     int64
location       object
dtype: object

In [57]:
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [58]:
data.dtypes

timestamp      datetime64[ns]
customer_no             int64
location               object
dtype: object

In [59]:
data.index

RangeIndex(start=0, stop=4884, step=1)

In [60]:
data.shape

(4884, 3)

In [61]:
data.isna().any()

timestamp      False
customer_no    False
location       False
dtype: bool

____

In [62]:
#total no of cust
data['customer_no'].nunique()

1447

In [63]:
#checking if all cust are "checked out"
data[data['location'] == 'checkout'].value_counts().count() # 10 left?
#data['location'].value_counts()

1437

In [64]:
#number of locations each cust visited
test = pd.DataFrame(data.groupby(by='customer_no')['location'].count())

#data_test = pd.DataFrame(data.groupby(by='customer_no')['location'].count())
#data_test['location']==1
#data_test['location'].unique()

In [65]:
test

,location
customer_no,
1,2
2,2
3,2
4,2
5,2
...,...
1443,2
1444,2
1445,1


In [66]:
#THIS FUNC DOESN'T COVER THE WHOLE RANGE OF CUSTOMERS

# func for checkout
def check_out (df):
    li = []
    
    for index, row in df.iterrows():
        if row['location'] == 'checkout':
            li.append(row['customer_no'])
    li.sort()
    #return li
    
    def find_missing(lst): 
        return [x for x in range(lst[0], lst[-1]+1) if x not in lst]
    
    return find_missing(li)
    

In [67]:
check_out(data)

[1430, 1433, 1437, 1439, 1440, 1441, 1443]

In [68]:
#THIS FUNC WORK'S

#here the customers without 'checkout'
for i in range(data['customer_no'].nunique()+1):
    data[data['customer_no'] == i][-1:]['location']
    if data[data['customer_no'] == i][-1:]['location'].all() != 'checkout':
        print(data[data['customer_no'] == i][-1:])
    else:
        pass


Empty DataFrame
Columns: [timestamp, customer_no, location]
Index: []
               timestamp  customer_no location
4875 2019-09-02 21:49:00         1430    fruit
               timestamp  customer_no location
4876 2019-09-02 21:49:00         1433    fruit
               timestamp  customer_no location
4863 2019-09-02 21:47:00         1437    dairy
               timestamp  customer_no location
4860 2019-09-02 21:46:00         1439    fruit
               timestamp  customer_no location
4877 2019-09-02 21:49:00         1440   spices
               timestamp  customer_no location
4878 2019-09-02 21:49:00         1441   spices
               timestamp  customer_no location
4873 2019-09-02 21:48:00         1443    dairy
               timestamp  customer_no location
4881 2019-09-02 21:49:00         1445    dairy
               timestamp  customer_no location
4882 2019-09-02 21:50:00         1446    dairy
               timestamp  customer_no location
4883 2019-09-02 21:50:00         1447

In [69]:
#new col with only hourly timestamp
data['time'] = data['timestamp'].dt.time

In [70]:
data.head()

,timestamp,customer_no,location,time
0,2019-09-02 07:03:00,1,dairy,07:03:00
1,2019-09-02 07:03:00,2,dairy,07:03:00
2,2019-09-02 07:04:00,3,dairy,07:04:00
3,2019-09-02 07:04:00,4,dairy,07:04:00
4,2019-09-02 07:04:00,5,spices,07:04:00


____

In [71]:
#Calculate the total number of customers in each section
data.groupby(by='location')['customer_no'].count().sort_values(ascending=False)

location
checkout    1437
fruit       1005
dairy        895
drinks       797
spices       750
Name: customer_no, dtype: int64

In [72]:
#Calculate the total number of customers in each section over time (per Minute)
data.groupby(by=['location', 'timestamp'])['customer_no'].count()

location  timestamp          
checkout  2019-09-02 07:05:00    2
          2019-09-02 07:06:00    2
          2019-09-02 07:07:00    1
          2019-09-02 07:08:00    3
          2019-09-02 07:09:00    1
                                ..
spices    2019-09-02 21:43:00    1
          2019-09-02 21:44:00    1
          2019-09-02 21:46:00    2
          2019-09-02 21:48:00    1
          2019-09-02 21:49:00    2
Name: customer_no, Length: 2797, dtype: int64

In [73]:
#Calculate the total number of customers in each section over time (per Hour)
pd.DataFrame(data.set_index('timestamp').groupby([pd.Grouper(freq='H'), pd.Grouper('location')])['customer_no'].count())

customer_no
timestamp           location             
2019-09-02 07:00:00 checkout           92
                    dairy              59
                    drinks             60
                    fruit              54
                    spices             52
...                                   ...
2019-09-02 21:00:00 checkout           52
                    dairy              22
                    drinks             28
                    fruit              33
                    spices             18

[75 rows x 1 columns]

In [74]:
#Display the number of customers at checkout over time (per Minute)
pd.DataFrame(data.groupby(by=['location', 'timestamp'])['customer_no'].count()).filter(like='checkout', axis=0)

customer_no
location timestamp                       
checkout 2019-09-02 07:05:00            2
         2019-09-02 07:06:00            2
         2019-09-02 07:07:00            1
         2019-09-02 07:08:00            3
         2019-09-02 07:09:00            1
...                                   ...
         2019-09-02 21:44:00            1
         2019-09-02 21:46:00            1
         2019-09-02 21:47:00            2
         2019-09-02 21:48:00            2
         2019-09-02 21:49:00            2

[700 rows x 1 columns]

In [75]:
#Display the number of customers at checkout over time (per Hour)
pd.DataFrame(data.set_index('timestamp').groupby(
    [pd.Grouper(freq='H'), pd.Grouper('location')])['customer_no'].count()).filter(like='checkout', axis=0)

,,customer_no
timestamp,location,
2019-09-02 07:00:00,checkout,92
2019-09-02 08:00:00,checkout,112
2019-09-02 09:00:00,checkout,85
2019-09-02 10:00:00,checkout,90
2019-09-02 11:00:00,checkout,55
2019-09-02 12:00:00,checkout,73
2019-09-02 13:00:00,checkout,120
2019-09-02 14:00:00,checkout,90
2019-09-02 15:00:00,checkout,94


In [76]:
#THIS FUNC DOESN'T TAKE CUST WITHOUT 'CHECKOUT' IN ACOUNT

#func for all customers
def count_time(df):
    df = df.copy()
    cust_no = 1
    d = {}
    while cust_no <= 1447:   
    
        result = []
        df = df.sort_values(['customer_no', 'timestamp'])
        
        ts = ((df[df['customer_no'] == cust_no]['timestamp'].iloc[-1]) - (df[df['customer_no'] == cust_no]['timestamp'].iloc[0])).seconds//60
        d[cust_no] = ts
        cust_no += 1
    return d
        

In [77]:
visit = pd.DataFrame.from_dict(count_time(data), orient='index')

In [78]:
visit.index.rename('customer_no', inplace=True)

In [79]:
visit.rename(columns={0: "minutes_spend"}, inplace=True)

In [80]:
visit

,minutes_spend
customer_no,
1,2
2,3
3,2
4,4
5,1
...,...
1443,1
1444,1
1445,0


____

In [81]:
# NEW COLS FOR before & after LOCATIONS
data['before'] = data['location']

data.sort_values(['customer_no', 'timestamp'], inplace=True)
data['after'] = data['before'].shift(-1)
data.tail(20)


,timestamp,customer_no,location,time,before,after
4853,2019-09-02 21:43:00,1437,fruit,21:43:00,fruit,spices
4855,2019-09-02 21:44:00,1437,spices,21:44:00,spices,dairy
4863,2019-09-02 21:47:00,1437,dairy,21:47:00,dairy,spices
4859,2019-09-02 21:46:00,1438,spices,21:46:00,spices,checkout
4864,2019-09-02 21:47:00,1438,checkout,21:47:00,checkout,fruit
4860,2019-09-02 21:46:00,1439,fruit,21:46:00,fruit,drinks
4861,2019-09-02 21:46:00,1440,drinks,21:46:00,drinks,spices
4877,2019-09-02 21:49:00,1440,spices,21:49:00,spices,dairy
4865,2019-09-02 21:47:00,1441,dairy,21:47:00,dairy,drinks
4872,2019-09-02 21:48:00,1441,drinks,21:48:00,drinks,spices


In [82]:
pd.crosstab(data['after'], data['before'], normalize=0).drop(columns=['checkout'])

before,dairy,drinks,fruit,spices
after,,,,
checkout,0.215727,0.296451,0.364649,0.123173
dairy,0.001119,0.025727,0.251678,0.263982
drinks,0.273526,0.000000,0.165621,0.272271
fruit,0.180100,0.185075,0.000995,0.119403
spices,0.246667,0.216000,0.162667,0.000000


In [103]:
data.index.get_loc(1)

2

In [120]:
#THIS FUNC WORK'S

#here the index of customers without 'checkout'
cl = []
for i in range(1,data['customer_no'].nunique()+1):
    data[data['customer_no'] == i][-1:]['location']
    if data[data['customer_no'] == i][-1:]['location'].all() != 'checkout':
        cl.append(i)
    else:
        pass

In [121]:
cl

[1430, 1433, 1437, 1439, 1440, 1441, 1443, 1445, 1446, 1447]

In [141]:
#getting all rows of the customers from cl
in_li = []
for i in cl: 
    ind = data[data['customer_no'] == i].index.to_list()
    in_li.append(ind)
    flat_list = [item for sublist in in_li for item in sublist]

[4840, 4846, 4848, 4868, 4875, 4843, 4856, 4870, 4876, 4853, 4855, 4863, 4860, 4861, 4877, 4865, 4872, 4878, 4867, 4873, 4881, 4882, 4883]


In [150]:
len(flat_list)

23

In [151]:
#dropping by index
data.drop(flat_list,axis=0, inplace=True)

In [149]:
data.shape #before 4884 / after dropping 4861, so -23

(4884, 6)

In [152]:
data.shape

(4861, 6)